<h3>
<center>
محمد‌مهدی ابوترابی
<br>
تمرین اول درس پردازش زبان‌های طبیعی
<br>
98105557
</center>
</h3>

In [1]:
# personal data
student_number = 98105557
Name = 'Mohammad Mahdi'
Last_Name = 'Abootorabi'

In [1]:
# install libraries
# !pip install nltk
# !pip install numpy
# !pip install pandas

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
دیتای مورد نظر از کراول کردن سایت skysport بدست آمده‌است.
در واقع این سایت احتمالا که همانطور می‌دانید سایتی ورزشی است که در دنیا بسیار معروف بوده و یک خبرگذاری می‌باشد که خبر‌های متفاوت ورزشی مخصوصا فوتبال را پوشش می‌دهد.
برای 6 کتگوری (ورزش) مختلف اخبار این سایت را کراول کردم که اسکریپت کراول آن را در کنار این نوتبوک می‌توانید مشاهده کنید. چالش‌های نسبتا زیادی در کراول به وجود آمد چون ساختار سایت برای بخش‌های مختلف تقریبا یکی نبود ولی در نهایت یک دیتای جیسون به تفکیک ورزش‌ها بدست آمده که اخبار همراه title آن‌ها در آن قرار دارد. 
و متن نهایی حاصل چیزی حدود 2.3 مگ شد.
</div>

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
نخست لایبرری‌های مورد نظر را لود می‌کنیم.
</div>

In [40]:
# libraries
import codecs
import os
import json
import tqdm
from collections import Counter
from nltk import FreqDist
import nltk
import random
import pandas as pd
import string
import numpy as np
import itertools
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import brown

In [41]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahdi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahdi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahdi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Mahdi\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Mahdi\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mahdi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]      

True

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
نخست دیتای مورد نظر که آن را کراول کرده‌ایم را باز می‌کنیم.
</div>

In [42]:
# read data
DIRNAME = 'skysports'

with open(f"{DIRNAME}/data.json", "r", encoding="utf-8") as text_file:
    data = json.loads(text_file.read())

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
ساختار دیتا را در زیر می‌توانید مشاهده کنید.
</div>

In [43]:
{k: len(v) for k,v in data.items()}

{'football': 154, 'f1': 62, 'cricket': 157, 'golf': 57, 'nfl': 47, 'darts': 32}

### normalization

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
دو تابع می‌نویسیم که اولی نرمالایز کردن را برای متن و دومی برای دیتا انجام می‌دهد.
</div>

In [44]:
# normalization
def normalize_text(text, minimum_length=1, stopword_removal=True, stopwords_domain=[], lower_case=True,
                       punctuation_removal=True):
    
    normalized_tokens = word_tokenize(text)

    if stopword_removal:
        # Remove stopwords in English and also the given domain stopwords
        stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
        domain_stopwords = [x.lower() for x in stopwords_domain]
        normalized_tokens = [word for word in normalized_tokens if word.lower() not in domain_stopwords + stopwords]

    if punctuation_removal:
        # Remove punctuations
        normalized_tokens = [word for word in normalized_tokens if word not in string.punctuation]

    if lower_case:
        # Convert everything to lowercase and filter based on a min length
        normalized_tokens = [word.lower() for word in normalized_tokens if len(word) > minimum_length]
    else:
        normalized_tokens = [word for word in normalized_tokens if len(word) > minimum_length]

    return normalized_tokens

In [45]:
def normalize_data(data, stopword_removal=True, stopwords_domain=[]):
    tokens_dict = {}
    for sport, texts in data.items():
        inner_tokens_dict = {}
        for title, text in texts.items():
            inner_tokens_dict[title] = normalize_text(text, stopword_removal=stopword_removal, stopwords_domain=stopwords_domain)
        tokens_dict[sport] = inner_tokens_dict
    return tokens_dict

In [46]:
tokens_dict = normalize_data(data, stopword_removal=False)

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
حال یک سمپل از یکی از متن‌ها پس از نرمالایز می‌بینیم.
</div>

In [49]:
" ".join(random.sample(list(tokens_dict['football'].values()),1)[0])[:1000]

"england captain harry kane has chosen world mental health day to launch the harry kane foundation hkf with long-term goal to help transform generation 's thinking about mental health the launch of his own dedicated foundation marks the start of kane 's journey to learn more about mental health and work with chosen charities and strategic partners hkf will use the tottenham striker 's profile and curate partnerships to reach audiences of all ages through tactical awareness campaigns and practical support the purpose of hkf is to help normalise conversations around mental health promote positive habits that support mental health and tackle stigma kane said `` am delighted to launch my foundation today on world mental health day it marks the start of my journey to learn more about mental health and use my position hopefully to encourage others to look after their mental health be their best and not be afraid to ask for help `` while 'm far from an expert on the subject 've learnt that th

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
تابعی که توکن‌های رشته‌های مختلف را با هم aggregate می‌کند تا بتوانیم تحلیل را به تفکیک نوع ورزش انجام بدهیم.
</div>

In [50]:
def create_aggregate_sports_tokens(tokens_dict):
    sports_dict = {}
    for sport, texts in tokens_dict.items():
        ls = []
        for tokens in texts.values():
            ls.extend(tokens)
        sports_dict[sport] = ls
    return sports_dict

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
تابعی برای درست کردن دیتا‌فریم مربوط به توکنایز‌کردن و تحلیل بسامد: 
</div>

In [51]:
def create_dataframe_freq(tokens_dict):
    return pd.DataFrame({key: FreqDist(value).most_common(20) for key,value in tokens_dict.items()})

In [52]:
create_dataframe_freq(create_aggregate_sports_tokens(tokens_dict))

,football,f1,cricket,golf,nfl,darts
0,"(the, 6572)","(the, 2612)","(the, 5961)","(the, 2136)","(the, 2547)","(the, 1208)"
1,"(to, 3424)","(to, 1334)","(to, 2841)","(to, 893)","(to, 1129)","(to, 776)"
2,"(and, 2861)","(and, 1035)","(and, 2763)","(and, 834)","(and, 870)","(in, 528)"
3,"(in, 2484)","(of, 894)","(in, 2479)","(of, 678)","(in, 813)","(and, 480)"
4,"(of, 2152)","(in, 804)","(of, 2093)","(in, 662)","(of, 677)","(of, 348)"
5,"('s, 1745)","(for, 752)","(for, 1743)","(on, 451)","(for, 610)","(for, 332)"
6,"(for, 1725)","(``, 727)","('s, 1550)","(for, 449)","(on, 576)","('s, 273)"
7,"(``, 1375)","('s, 640)","(``, 1306)","(``, 404)","('s, 522)","(``, 261)"
8,"(it, 1369)","(it, 630)","(with, 1198)","('s, 382)","(``, 477)","(his, 239)"
9,"(is, 1277)","(that, 561)","(on, 1098)","(with, 327)","(that, 455)","(it, 238)"


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
همانطور که می‌بینید چون هنوز stopwords
    را اثر نداده‌ایم از تحلیل بسامد اطلاعات ارزشمندی نصیبمان نمی‌شود.
    و چیز‌هایی مثل حرف اضافه و نقطه و ... بیش‌ترین فرکانس را دارند.
</div>

### stopwords

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
تحلیل بسامد با در نظر‌گرفتن stopword 
    های general زبان انگلیسی
</div>

In [53]:
tokens_dict = normalize_data(data)
create_dataframe_freq(create_aggregate_sports_tokens(tokens_dict))

,football,f1,cricket,golf,nfl,darts
0,"('s, 1745)","(``, 727)","('s, 1550)","(``, 404)","('s, 522)","('s, 273)"
1,"(``, 1375)","('s, 640)","(``, 1306)","('s, 382)","(``, 477)","(``, 261)"
2,"('', 727)","('', 452)","(england, 853)","('', 253)","('', 296)","(world, 209)"
3,"(league, 659)","(verstappen, 270)","('', 655)","(golf, 243)","(nfl, 291)","(van, 205)"
4,"(sky, 590)","(use, 263)","(cricket, 530)","(tour, 215)","(game, 232)","(player, 184)"
5,"(sports, 544)","(race, 256)","(player, 504)","(team, 188)","(player, 202)","(please, 158)"
6,"(player, 369)","(video, 254)","(use, 412)","(please, 176)","(use, 176)","(use, 158)"
7,"(n't, 366)","(please, 253)","(please, 401)","(player, 176)","(please, 175)","(chrome, 158)"
8,"(football, 358)","(chrome, 253)","(video, 401)","(liv, 164)","(video, 175)","(browser, 158)"
9,"(players, 344)","(browser, 253)","(chrome, 400)","(use, 152)","(chrome, 174)","(accessible, 158)"


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
هنوز همانطور که می‌بینید مشکلاتی وجود دارد و کلمات زیادی است که به درستی حذف نشده‌اند.
    پس یک فایل custom_stopwords
    خودمان درست می‌کنیم و در آن چیز‌هایی که در نظر داریم حذف‌شوند و مخصوص این دامنه هستند را اضافه می‌کنیم.
</div>

In [54]:
custom_stop_words = [x.strip() for x in codecs.open('skysports/custom_stopwords.txt','r','utf-8').readlines()]
custom_stop_words

['sky',
 'sports',
 "n't",
 "'s",
 '``',
 "''",
 'please',
 'chrome',
 'accessible',
 'browser',
 'video']

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
تحلیل بسامد با در نظر‌گرفتن stopword 
    های جدید:
</div>

In [55]:
tokens_dict = normalize_data(data, stopwords_domain=custom_stop_words)
create_dataframe_freq(create_aggregate_sports_tokens(tokens_dict))

,football,f1,cricket,golf,nfl,darts
0,"(league, 659)","(verstappen, 270)","(england, 853)","(golf, 243)","(nfl, 291)","(world, 209)"
1,"(player, 369)","(use, 263)","(cricket, 530)","(tour, 215)","(game, 232)","(van, 205)"
2,"(football, 358)","(race, 256)","(player, 504)","(team, 188)","(player, 202)","(player, 184)"
3,"(players, 344)","(player, 253)","(use, 412)","(player, 176)","(use, 176)","(use, 158)"
4,"(game, 332)","(red, 208)","(first, 340)","(liv, 164)","(season, 165)","(gerwen, 125)"
5,"(play, 298)","(said, 174)","(game, 331)","(use, 152)","(week, 136)","(darts, 117)"
6,"(back, 292)","(bull, 168)","(world, 299)","(world, 145)","(first, 129)","(price, 105)"
7,"(season, 287)","(car, 165)","(said, 295)","(cup, 140)","(sunday, 124)","(first, 103)"
8,"(get, 285)","(f1, 153)","(test, 275)","(one, 124)","(yards, 118)","(set, 98)"
9,"(said, 283)","(hamilton, 146)","(t20, 274)","(players, 119)","(two, 112)","(leg, 79)"


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
    اطلاعات زیادی را حالا می‌توان از این خروجی برداشت کرد. در هر کتگوری می‌توان فهمید که چه کلماتی پرتکرارتر و مهم‌تر هستند و معمولا در اخبار مربوط به آن رشته زیاد استفاده می‌شوند. برای مثال در ورزش f1 که
    مربوط به فرمول یک می‌باشد پرتکرار ترین کلمه همانطور که می‌بینید verstappen می‌باشد.
    که خب verstappen معروف‌ترین فرد در این رشته و رنک اول این مسابقات می‌باشد که این نشان می‌دهد که در تعداد اخبار زیادی از او نام برده شده است.
<br>
یا برای مثال در رشته‌ی کریکت از آن‌جا که در اصل skysports متعلق به کشور انگلیس است و این رشته
    بیشتر در این کشور بازی می‌شود و کامیونیتی جهانی آن به قدرت بقیه رشته‌ها نیست کلمه‌ی england .بیشترین فرکانس تکرار را دارد
که این هم بسیار منطقی است و به نظر بیشتر اخبار این رشته تنها مربوط به خود انگلیس و لوکال می‌باشد.
</div>

### Lemmatization/Stemming

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
حال سعی می‌کنیم به کمک 
    Lemmatization/Stemming
    اطلاعات ما بهتر و کاربردی‌تر شوند و با بردن کلمات به ریشه‌ی مشترک به اطلاعات بهتری برسیم.
</div>

In [56]:
lemmatizer = WordNetLemmatizer()
stemmer  = PorterStemmer()


def get_lemma_set(tok, opt=1):
    if opt ==1:
        return stemmer.stem(tok)
    if opt ==2:
        return lemmatizer.lemmatize(tok)

In [57]:
opt = 2

sports_aggregate_tokens_dict = create_aggregate_sports_tokens(tokens_dict)
sports_aggregate_tokens_dict_after_lemma = {key: [get_lemma_set(word, opt) for word in value] for key,value in sports_aggregate_tokens_dict.items()}
create_dataframe_freq(sports_aggregate_tokens_dict_after_lemma)

,football,f1,cricket,golf,nfl,darts
0,"(player, 713)","(race, 319)","(england, 853)","(player, 295)","(nfl, 291)","(player, 221)"
1,"(league, 670)","(verstappen, 270)","(player, 670)","(golf, 243)","(game, 288)","(world, 211)"
2,"(game, 542)","(use, 263)","(cricket, 530)","(tour, 220)","(player, 230)","(van, 205)"
3,"(goal, 384)","(player, 253)","(game, 441)","(team, 196)","(use, 176)","(use, 158)"
4,"(football, 358)","(car, 227)","(ball, 440)","(liv, 164)","(season, 176)","(leg, 141)"
5,"(play, 305)","(driver, 210)","(use, 412)","(use, 152)","(week, 169)","(dart, 129)"
6,"(get, 301)","(red, 208)","(first, 340)","(cup, 147)","(team, 148)","(gerwen, 125)"
7,"(time, 297)","(team, 195)","(test, 305)","(world, 145)","(first, 129)","(set, 107)"
8,"(season, 296)","(said, 174)","(world, 300)","(one, 136)","(sunday, 124)","(price, 105)"
9,"(club, 296)","(bull, 170)","(said, 295)","(event, 119)","(quarterback, 123)","(first, 103)"


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
خب همانطور که می‌بینید نتایج نسبت به قسمت قبل باز هم دست‌خوش تغییر شد.
    الان باز کمی منطقی‌تر به نظر می‌رسند. در رشته‌ی فوتبال الان کلمه‌ی player بیشترین استفاده را دارد و پس از آن کلمه‌ی لیگ که هر دوی این‌ها بسیار منطقی هستند.
     <br>
    در رشته‌ی f1 که در بالا هم آن را تحلیل کردیم حالا بعد از کمک گرفتن از lemmatize کردن همانطور که می‌بینید کلمه‌ی race بیشترین استفاده را دارد که این هم بسیار منطقی است چون منطقا در این رشته این کلمه بسیار استفاده می‌شود.
    <br>
    در رشته‌ی کریکت همچون قبل کلمه‌ی انگلستان بیشترین فرکانس استفاده را دارد. علاوه بر آن کلمات player و cricket و game هم زیاد استفاده شدند که به نظر درست هم هست.
    <br>
   در ورزش گلف هم کلمات player و golf بیشترین را دارند که این دو اصلا دور از ذهن نیستند و با حدس‌هایمان جور در می‌آید. همچنین کلمه‌ی
    tour پس از آن‌ها قرار دارد که این کلمه همچون کلمه‌ی لیگ در رشته‌ی فوتبال است و در این رشته معلوم است که بسیار از آن استفاده می‌شود و برای بیان کردن مسابقات مختلف از آن استفاده می‌شود.
    <br>
    در رشته‌ی nfl نیز چند کلمه‌ی اول همان چیز‌های typical هستند که همه انتظارشان را داریم.
    نکته‌ی جالب کلمه‌ی field است که در جزو کلمات پرتکرار است در این رشته و علت آن هم این است که به زمینی که مسابقات این رشته در آن انجام می‌شوند این کلمه گفته می‌شود.
    <br>
    در نهایت در رشته‌ی دارت نیز کلمات طبق انتظارمان هستند. نکته‌ی جالب کلمه‌ی Leg می‌باشد که به هر دست بازی در دارت گفته می‌شود و جزو کلمات پرتکرار در اخبار این رشته می‌باشد.
</div>

---

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
حال در زیر سعی می‌کنیم اطلاعات کلی راجع به مجموعه متن که بررسی کردیم به‌دست آوریم که در زیر آن را می‌بینید. 
</div>

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
نخست در زیر تحلیل بسامد را به طور کلی مشاهده می‌کنید.
</div>

In [58]:
general_freq_data = []
for value in sports_aggregate_tokens_dict_after_lemma.values():
    general_freq_data.extend(value)
freq_analysis_general = pd.DataFrame({"general freq" : FreqDist(general_freq_data).most_common(10)})  
freq_analysis_general

,general freq
0,"(player, 2382)"
1,"(use, 1406)"
2,"(game, 1375)"
3,"(england, 1154)"
4,"(team, 1041)"
5,"(said, 996)"
6,"(first, 987)"
7,"(world, 971)"
8,"(year, 852)"
9,"(two, 851)"


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
همانطور که انتظار می‌رفت واژه player پر‌کاربردترین است.
    با اختلاف زیادی هم پرتکرار ترین است چون در تمام ورزش‌ها تقریبا استفاده می‌شود.
    کلمات دیگری همچون "team" و "game" هم تکرار بسیار بالایی داشتند.
    که این‌ها هم مطابق انتظارمان هستند. کلمه‌ی england هم چون این نشریه انگلیسی هست منطقی می‌باشد.
    نکته‌ی جالب کلمه‌ی use است که دومین کلمه‌ی پرتکرار است و این همه استفاده شده است که بسیار جالب است.
</div>

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
در زیر اطلاعات کلی دیگری را راجع به متون مورد بررسی مشاهده می‌کنید.
</div>

In [59]:
total_words = []
for tokens in sports_aggregate_tokens_dict.values():
    total_words.extend([token for token in tokens if 'skysports' not in token])
    
print(f"Number of words is {len(total_words)}")
print(f"Number of unique words is {len(set(total_words))}")
print(f"Average word length is {np.sum([len(word) for word in total_words]) / len(total_words)}")
print(f"Longest word is {total_words[np.argmax([len(word) for word in total_words])]}")

Number of words is 207110
Number of unique words is 17883
Average word length is 5.908087489739752
Longest word is matchplayquarter-finalslisa


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
حال این اطلاعات را به طریق دیگری به تفکیک کتگوری‌ها مشاهده می‌کنیم.
</div>

In [60]:
sports_words_data =  {sport : [] for sport in data.keys()}
for sport, tokens in tqdm.tqdm(sports_aggregate_tokens_dict_after_lemma.items()):
    tokens = [token for token in tokens if '.com' not in token]
    sports_words_data[sport].append(len(data[sport]))
    sports_words_data[sport].append(len(tokens))
    sports_words_data[sport].append(len(set(tokens)))
    sports_words_data[sport].append(np.sum([len(word) for word in tokens]) / len(tokens))
    sports_words_data[sport].append(tokens[np.argmax([len(word) for word in tokens])])
sports_df = pd.DataFrame(sports_words_data, index=[
    'Number of news', 'Number of words', 'Number of unique words','Average word length', 'Longest word'])
sports_df  

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 68.97it/s]


,football,f1,cricket,golf,nfl,darts
Number of news,154,62,157,57,47,32
Number of words,68003,24489,61465,18950,21155,13059
Number of unique words,8681,3667,7512,3355,4494,2488
Average word length,5.911166,5.906448,5.758057,5.647757,5.739494,5.66636
Longest word,quicker-than-anticipated,hugely-controversial,all-around-the-ground,tougher-than-anticipated,possession-controlling,matchplayquarter-finalslisa


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
همانطور که می‌بینید تعداد کلمات یونیک استفاده شده در ورزش فوتبال بیشتر از بقیه است و همچنین هم کلمات طولانی‌تری به طور میانگین به کار رفته است و هم کلا تعداد کلمات بیشتری  داریم که در مقایسه با کریکت که حتی تعداد اخبار آن بیشتر است نشان از این دارد که ورزش فوتبال تخصصی‌تر و حرفه‌ای تر است یا حداقل این نشریه به صورت تخصصی‌تری به آن می‌پردازد.
</div>

---

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
در زیر تلاش می‌کنیم تا اطلاعات ارزشمند دیگری راجع به کلمات و جملات به تفکیک رشته بدست آوریم.
 Sentence tokenization در این بخش از 
هم استفاده کرده‌ایم.
</div>

In [61]:
sports_sentences_data = {sport : [] for sport in data.keys()}
for sport, texts in tqdm.tqdm(data.items()):
    for text in texts.values():
        sports_sentences_data[sport].extend(sent_tokenize(text))
print(f"Number of sentences is {len(list(itertools.chain(*(list(sports_sentences_data.values())))))}")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 11.56it/s]

Number of sentences is 15883


In [62]:
sports_texts_data =  {sport : [] for sport in data.keys()}

for sport, sentences in tqdm.tqdm(sports_sentences_data.items()):
    sports_texts_data[sport].append(len(sentences) / len(data[sport]))
    sports_texts_data[sport].append(np.average([len(sent) for sent in sentences]))

for sport, tokens in tqdm.tqdm(sports_aggregate_tokens_dict.items()):
    sports_texts_data[sport].append(len(tokens) / len(data[sport]))
    sports_texts_data[sport].append(np.average([len(word) for word in tokens]))
sports_df = pd.DataFrame(sports_texts_data, index=[
    'Average number of sentences in news', 'Average sentence length','Average number of words in news', 'Average words length'])
sports_df

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 206.70it/s]


,football,f1,cricket,golf,nfl,darts
Average number of sentences in news,37.545455,29.193548,28.191083,23.649123,34.234043,28.375000
Average sentence length,126.357662,150.217127,144.894939,143.233680,141.336234,147.111233
Average number of words in news,441.675325,394.983871,391.636943,332.473684,450.744681,408.281250
Average words length,6.019627,5.994161,5.857059,5.748826,5.877508,5.744508


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
همانطور که می‌بینید تعداد جملات در رشته‌ی فوتبال نسبت به بقیه‌ی رشته‌ها بیشتر است. همچنین طول جملات کوتاه‌تر از بقیه‌ی رشته‌ها است. در واقع متون فوتبال جملات زیاد ولی کوتاه هستند. با این حال بیشترین تعداد کلمات را در متن دارند و می‌توان نتیجه گرفت که به نسبت طولانی‌ترین اخبار مال کتگوری فوتبال می‌باشد.
    از طرف دیگر اخبار گلف کوتاه‌ترین اخبار می‌باشند و تعداد جملات کمتری در هر متن آن قابل مشاهده است ولی هر جمله ی آن نسبتا کوتاه‌تر است. همچنین فوتبال از کلمات طولانی‌تر و گلف از کلمات کوتاه‌تر بهره می‌برد.
</div>

### POS-tags

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
در این قسمت به کمک POS-tagging نخست یک tagger را train می‌کنیم.
    آن را بر روی یک دیتاست news که مربوط به brown می‌باشد train می‌کنیم.
</div>

In [63]:
def create_data_sets(sentences):
    size = int(len(sentences) * 0.9)
    train_sents = sentences[:size]
    test_sents = sentences[size:]
    return train_sents, test_sents

def build_backoff_tagger (train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    return t2


def train_tagger(already_tagged_sents):
    train_sents, test_sents = create_data_sets(already_tagged_sents)
    ngram_tagger = build_backoff_tagger(train_sents)
    print ("%0.3f pos accuracy on test set" % ngram_tagger.evaluate(test_sents))
    return ngram_tagger

In [64]:
brown_news_tagged = brown.tagged_sents(categories='news', tagset='universal',)
brown_tagger = train_tagger(brown_news_tagged)

0.849 pos accuracy on test set


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
    همانطور که می‌بینید به accuracy 85% رسیدیم.
</div>

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
    یک جمله‌ی رندم از بخش فوتبال را با آن tag می‌زنیم.
</div>

In [65]:
brown_tagger.tag(word_tokenize(sports_sentences_data['football'][0]))

[('All', 'PRT'),
 ('you', 'PRON'),
 ('need', 'VERB'),
 ('to', 'PRT'),
 ('know', 'VERB'),
 ('on', 'ADP'),
 ('how', 'ADV'),
 ('to', 'PRT'),
 ('watch', 'VERB'),
 ('highlights', 'NN'),
 ('this', 'DET'),
 ('midweek', 'NN'),
 ('from', 'ADP'),
 ('every', 'DET'),
 ('Premier', 'NOUN'),
 ('League', 'NOUN'),
 ('game', 'NOUN'),
 ('with', 'ADP'),
 ('Sky', 'NN'),
 ('Sports', 'NOUN'),
 (',', '.'),
 ('including', 'ADP'),
 ('Man', 'NOUN'),
 ('Utd', 'NN'),
 ('vs', 'NN'),
 ('Tottenham', 'NN'),
 ('.', '.')]

In [66]:
sports_sentences_data['football'][0]

'All you need to know on how to watch highlights this midweek from every Premier League game with Sky Sports, including Man Utd vs Tottenham.'

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
    حال سعی می‌کنیم تا به کمک آن صفت‌ها را پیدا کنیم و برای هر ورزش دیتافریم تکرار صفات درون آن را بدست آوریم.
</div>

In [67]:
df_advjj = {}
opt = 2

for sport, sentences in tqdm.tqdm(sports_sentences_data.items()):
    selected = []
    for sentence in sentences:
        tokens = normalize_text(sentence, stopwords_domain=custom_stop_words)
        tokens = [get_lemma_set(word, opt) for word in tokens]
        tagged_sentence = brown_tagger.tag(tokens)
        for w, pos in tagged_sentence:
            if pos=='ADJ':
                selected.append(w) 
    df_advjj[sport]= list(FreqDist(selected).most_common(20))
df_advjj = pd.DataFrame(df_advjj)    
df_advjj

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.50s/it]


,football,f1,cricket,golf,nfl,darts
0,"(last, 235)","(red, 208)","(first, 340)","(last, 88)","(first, 129)","(first, 103)"
1,"(first, 227)","(last, 103)","(final, 198)","(first, 85)","(last, 81)","(final, 78)"
2,"(free, 209)","(first, 103)","(last, 190)","(best, 72)","(new, 79)","(grand, 56)"
3,"(super, 192)","(second, 93)","(second, 180)","(next, 66)","(second, 56)","(second, 53)"
4,"(top, 139)","(next, 60)","(international, 172)","(international, 63)","(giant, 56)","(last, 51)"
5,"(new, 138)","(grand, 58)","(new, 147)","(final, 58)","(great, 40)","(top, 43)"
6,"(second, 134)","(final, 53)","(best, 104)","(open, 48)","(super, 38)","(next, 35)"
7,"(better, 132)","(new, 43)","(great, 100)","(second, 48)","(good, 37)","(best, 35)"
8,"(final, 99)","(much, 40)","(good, 98)","(third, 37)","(fourth, 34)","(third, 31)"
9,"(best, 97)","(good, 39)","(third, 97)","(top, 37)","(defensive, 34)","(brilliant, 23)"


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
    خب همانطور که می‌بینید صفات پرتکرار در هر رشته را مشاهده می‌کنید. در اکثر رشته‌ها دو صفت last و first بسیار پرتکرار هستند که منطقی هم است و همین انتظار هم می‌رود.
    در f1 بسیار جالب است که صفت red بسیار تکرار شده است و با اختلاف اول است که احتمالا به رنگ ماشین‌ها که اکثرا قرمز است اشاره دارد و زیاد در متون تکرار می‌شود.
        همچنین second نیز در تمام رشته‌ها جزو کلمات پرتکرار است. با بررسی این دیتافریم اطلاعات ارزشمند دیگری نیز می‌توان پیدا کرد.
</div>

---

### Automated readability index

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
در این بخش نهایی به ویژگی‌های دیگری از متون می‌پردازیم. در این بخش این index را برای کل متون به صورت جنرال بدست می‌آوریم.
    همانطور که در سر کلاس گفته شد این index معیاری از پیچیدگی متون است که می‌خواهیم ببینیم برای این سایت به صورت کلی پیچیدگی متون از چه قرار است.
</div>

In [68]:
# Automated readability index
# https://en.wikipedia.org/wiki/Automated_readability_index
# general
len_general_words = 0
len_general_characters = 0
for sport, tokens in sports_aggregate_tokens_dict.items():
    len_general_words += len(tokens)
    for token in tokens:
        len_general_characters += len(token)
len_general_sentences = len(list(itertools.chain(*(list(sports_sentences_data.values())))))
general_ari = (4.71 * (len_general_characters / len_general_words)) + (0.5 * (len_general_words / len_general_sentences)) - 21.43
print(f'Automated readability index: {general_ari}')

Automated readability index: 12.936767723704122


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
طبق عدد بدست آمده که حدود 13 است این نوشته‌ها و کلا این سایت برای افراد 17 سال به بالا مناسب است.
    که این با منطق ما هم سازگار است و چون تخصصی راجع به ورزش‌های گوناگون است کلمات ساده‌ای در آن به کار نرفته است که برای کودکان هم مناسب باشد و نسبتا با متون پیچیده‌ای سر و کار داریم.
</div>

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
حال همین تحلیل را به تفکیک کتگوری انجام می‌دهیم.
</div>

In [69]:
ari_sports = {}
for sport, tokens in sports_aggregate_tokens_dict.items():
    len_words = len(tokens)
    len_characters = 0
    for token in tokens:
        len_characters += len(token)
    len_sentences = len(sports_sentences_data[sport])
    ari_sports[sport] = (4.71 * (len_characters / len_words)) + (0.5 * (len_words / len_sentences)) - 21.43
ari_sports

{'football': 12.804318695717505,
 'f1': 13.56741375821234,
 'cricket': 13.102862778844042,
 'golf': 12.676272751357672,
 'nfl': 12.836342669486733,
 'darts': 12.82101701421729}

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
طبق نتایج بدست آمده اخبار مربوط به f1 و cricket متون پیچیده‌تری هستند که این با منطق ما هم از نوعی سازگار است.
    البته باز به طور کلی خیلی شبیه به هم هستند کتگوری‌های مختلف اما باز f1 از همه پیچیده‌تر است.
    که در زندگی روزمره هم اگر ببینیم چون خیلی اصطلاحات سخت و پیچیده‌ای دارد و کلا خیلی خیلی تخصصی‌تر از بقیه است به نظر منطقی می‌آید.
</div>

### Coleman–Liau index

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
این بار با یک معیار دیگر پیچیدگی متون را اندازه می‌گیریم.
</div>

In [70]:
# Coleman–Liau index
# https://en.wikipedia.org/wiki/Coleman%E2%80%93Liau_index
general_cli = (5.88 * (len_general_characters / len_general_words)) - (29.6 * (len_general_sentences / len_general_words)) - 15.8
print(f'Coleman–Liau index: {general_cli}')

Coleman–Liau index: 16.69189217886532


<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
عدد 16.7 عدد بالایی برای این معیار است و برای افراد بزرگسال و به صورت تخصصی این متون را پیشنهاد می‌دهد که از نگاه بیرون هم نظر خودمان همین است.
</div>

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
حال همین تحلیل را به تفکیک کتگوری انجام می‌دهیم.
</div>

In [71]:
cli_sports = {}
for sport, tokens in sports_aggregate_tokens_dict.items():
    len_words = len(tokens)
    len_characters = 0
    for token in tokens:
        len_characters += len(token)
    len_sentences = len(sports_sentences_data[sport])
    cli_sports[sport] = (5.88 * (len_characters / len_words)) - (29.6 * (len_sentences / len_words)) - 15.8
cli_sports

{'football': 17.079203152106796,
 'f1': 17.257906815304832,
 'cricket': 16.508820401060387,
 'golf': 15.897624399767817,
 'nfl': 16.511626150578234,
 'darts': 15.920548029085342}

<div style="direction:rtl;font-family:XB Nazanin;font-size:16px;">
با این معیار باز هم f1 پیچیده‌ترین متون را دارد و golf هم سبک ترین و راحت‌ترین متون را دارد.
    در واقع با توجه به این دو index که بررسی کردیم می‌توان گفت که این دو همدیگر را تایید کردند و نتایجی که بالا گفتیم با اطمینان خوبی برقرار است.
</div>